In [1]:
import numpy as np

### Define vehicles and tasks sets

In [148]:
vehicles = [0,1,2,3]
tasks = [0,1,2]

### Create random utility matrix

We use a matrix to store the utilities of the vehicles for the different possible allocations.
The advantage of using a matrix compared to storing all the possible combinations in a dictionary, a hashmap or similar, is that we save a lot of space because we don't have to store the keys, and the acces time is the same. The other advantage is that we can loop easily on the matrix and use optimized numpy functions over this matrix. Using a matrix could also be interesting to move the computations on GPU to speed it up for high dimentionnal instances.

In [149]:
# Clean version :

def getShape(nb_vehicles, nb_tasks):
    """ Compute the shape for this settings
    
    Parameters :
        nb_vehicles : int, the number of vehicles
        nb_tasks : int, the number of tasks
        
    Returns :
        a tuple containing the shape (tasks^vehicles * vehicles),
        that is the shape for the utility matrix.
        
    """
    list_dim = [nb_tasks]*nb_vehicles +[nb_vehicles]# matrix of shape tasks^vehicles * vehicles
    return tuple(list_dim)

shapes = getShape(len(vehicles),len(tasks))
shapes

(3, 3, 3, 3, 4)

In [150]:
ut = np.random.randint(0,10,shapes) # create utility matrix with random utilities

In [155]:
ut[0][0][0][0] # get utility of all agents when all do task 0

array([6, 0, 9, 4])

In [156]:
ind = tuple([0,0,0,0]) # accessing the matrix from array as index
ut[ind]

array([6, 0, 9, 4])

In [157]:
ut[0][1][2][2] # get utility of all agents when allocation for vehicles to task is = 0,1,2,2 
# (means vehicule 0 do task 0, vehicule 1 do task 1, vehicule 2 do task 2, vehicule 3 do task 2)

array([9, 9, 5, 8])

### Defining usefull functions

In [137]:
def replaceAlloc(allocation, v, t):
    """ Compute the new allocation with task t asigned to vehicle v
    
    Parameters :
        allocation : List(int) the list of tasks allocades to each vehicle (in order)
        v : int, the vehicle id
        t : int, the task id
    
    Returns :
        List(int), the new allocation
    
    """
    return list(allocation[:v])+[t]+list(allocation[v+1:])

In [152]:
def is_EN(utilities, allocation, vehicles, tasks):
    """ Check if the current allocation is a Nash Equilibrium or not
    
    Parameters :
        utilities : Matrix(int) the utility matrix of dimension nb_tasks^nb_vehicles * nb_vehicles
        allocation : List(int) the list of tasks allocades to each vehicle (in order)
        vehicle : List(int) the list of vehicle ids
        tasks : List(int) the list of task ids
    
    Returns :
        Tuple(boolean, int)
        A tuple containing a boolean (True if this allocation is a Nash Equilibrium, else False)
        and an integer that is the id of a vehicle that can increase its utility
        by changing unilateraly its allocation (if not EN, -1)
    
    """
    for v in range(len(vehicles)) : # for each vehicle
        current_task = allocation[v]
        current_utility = utilities[tuple(allocation)][v]
        for t in range(len(tasks)) :
            if t != current_task : # check all other tasks
                temp_ind = replaceAlloc(allocation, v, t) # allocating task t to vehicle v
                utility = utilities[tuple(temp_ind)][v]
                if utility > current_utility : # changing to another task gives more utility -> Not NE
                    return (False, v)
    return (True, -1)

We also return the id of one vehicle that can increase its utility by changing its allocation, if the solution is not a Nash Equilibrium. 
It didn't increase the computation cost and avoids looping another time later on the utility table to find one in the Best Response Dynamics, it's all benefits.

##### Nash Equilibrium test example on small dimension

In [224]:
# Create setup : 2 vehicles, 3 tasks
v = [0,1] # don't change, it's 2D example
t = [0,1,2]
ut_test = np.random.randint(0,10,(len(t),len(t),2))
ut_test

array([[[0, 4],
        [3, 7],
        [7, 2]],

       [[9, 2],
        [2, 0],
        [8, 8]],

       [[8, 2],
        [2, 8],
        [9, 1]]])

In [225]:
# Check EN for allocation (0,0)
alloc = [0,0] # set allocation to check (0,1) -> first vehicle do task 0 and second do task 1
is_EN(ut_test, alloc, v, t) # random example

(False, 0)

In [226]:
alloc = [0,1]
ut_test[tuple(alloc)] = [0,0] # set allocation (0,1) to the lowest value for each vehicle
ut_test[2,1] = [1,0] # set another allocation to a better score for one vehicle (in case matrix is full zero)
is_EN(ut_test,alloc , v, t) # -> Is necessarly not an EN (result must be False)

(False, 0)

In [227]:
alloc = [0,1]
ut_test[tuple(alloc)] = [10,10] # set allocation (0,1) to the highest value for each vehicle
is_EN(ut_test, alloc, v, t) # -> Is necessarly  an EN (result must be True)

(True, -1)

In [228]:
ut_test_z = np.zeros((len(t),len(t),2)) # set all matrix to 0 (same value everywhere)
ut_test_z

array([[[0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.]]])

In [229]:
alloc = [2,1] # when Zero everywhere, no solution is strictly better than the current
is_EN(ut_test_z, alloc, v, t) # -> Is necessarly an EN (result must be True)

(True, -1)

### Best Response Dynamics

In [141]:
def getBestTask(utilities, allocation, v, tasks):
    """ Compute the best task for vehicle v
    
    Parameters :
        utilities : Matrix(int) the utility matrix of dimension nb_tasks^nb_vehicles * nb_vehicles
        allocation : List(int) the list of tasks allocades to each vehicle (in order)
        v : int, the vehicle id
        tasks : List(int) the list of task ids
        
    Returns : 
        int, the best task for vehicle v
    """
    best = np.argmax([utilities[tuple(replaceAlloc(allocation, v, t))][v] for t in range(len(tasks))])
    return best

In [214]:
def bestResponseDynamic(utilities, vehicles, tasks, maxsteps):
    """ Try to compute a Nash Equilibrium allofaction using Best Response Dynamics
    
    Parameters :
        utilities : Matrix(int) the utility matrix of dimension nb_tasks^nb_vehicles * nb_vehicles
        vehicles : List(int), the list of vehicle ids
        tasks : List(int) the list of task ids
        maxteps : int, the steps limit of the algorithm
        
    Returns : 
        List(int), a Nash Equilibrium allocation if one was found (no guarantee)
        
    """
    allocation = np.random.randint(0,len(tasks),len(vehicles)) # initial random allocation
    end, id_change = is_EN(utilities, allocation, vehicles, tasks)
    steps = 0
    while not(end) and steps < maxsteps:
        # vehicle id_change has interest to change to a better allocation
        best = getBestTask(utilities, allocation, id_change, tasks) # get its best unilateral allocation
        allocation = replaceAlloc(allocation, id_change, best) # set next allocation for id_change
        end, id_change = is_EN(utilities, allocation, vehicles, tasks)
        steps += 1
    if not(end) and steps >= maxsteps: # cut the exection if maxsteps reached
        print("Execution stopped : maximum step overflowed, no EN found.")
    return allocation

##### Best Response Dynamics test example on the initial matrix

In [280]:
ut[2,2,1,1] = [10,10,10,10] # we create a global optimal affectation -> At least one EN in utilities table

In [292]:
bestResponseDynamic(ut, vehicles, tasks, 1000) # the global optimum is reached (NE)

[2, 2, 1, 1]

In [291]:
bestResponseDynamic(ut, vehicles, tasks, 1000) # another Nash Equilibrium is sometimes reached

[0, 2, 0, 2]

### Fictitious Play

In [10]:
shapes = getShape(len(vehicles),len(tasks))
print(shapes)
ut = np.random.randint(0,10,shapes) # create utility matrix with random utilities

(3, 3, 3, 3, 4)


In [6]:
def computeFrequency(proposals, tasks):
    """ Compute the empirical frequency of each proposal
    
    Parameters :
        proposals : List(int), the allocation proposed
        tasks : List(int) the list of task ids
        
    Returns : 
        List(float), the empirical frequency of each proposal
    
    """
    return [np.count_nonzero(np.array(proposals) == t)/len(proposals) for t in range(len(tasks))]

In [19]:
# testing frequencies computation
proposals = [0,1,2,1,1,2,0,0,1,2,2,1,2,0,1,1,2,1,2,0,1] # proposals sequence made by an agent
taks = [0,1,2] # define tasks

In [22]:
computeFrequency(proposals,tasks) # compute frequency of each task in proposals

[0.23809523809523808, 0.42857142857142855, 0.3333333333333333]

In [35]:
def computeFullFrequencyMatrix(frequencies, vehicles, tasks): # not used
    """
        Compute the joined frequency matrix of proposed allocation for each vehicle
    """
    shapes = getShape(len(vehicles),len(tasks))
    fMat = np.zeros(shapes)
    allAlloc = [x for x,_ in np.ndenumerate(ut)]
    for alloc in allAlloc :
        fMat[alloc] = np.prod([frequencies[v][alloc[v]] for v in range(len(vehicles))])
    return fMat

In [36]:
# testing frequency matrix computation
frequencies = np.random.random((4,10)) # random frequencies

In [37]:
frequencies

array([[0.43819445, 0.02242756, 0.75844368, 0.34672042, 0.84612548,
        0.63451129, 0.98444621, 0.92280188, 0.96361494, 0.62077178],
       [0.34867839, 0.66745858, 0.15330373, 0.41385088, 0.41217432,
        0.93827241, 0.81652892, 0.39361675, 0.39504645, 0.81168782],
       [0.09905056, 0.30935464, 0.63764402, 0.28090354, 0.04137854,
        0.59448294, 0.41146702, 0.76507212, 0.61917395, 0.35979492],
       [0.99832658, 0.62650982, 0.44820006, 0.44499181, 0.29123082,
        0.08235175, 0.8698987 , 0.12316711, 0.70671548, 0.07297071]])

In [38]:
computeFullFrequencyMatrix(frequencies, vehicles, tasks)

array([[[[[1.51085040e-02, 1.51085040e-02, 1.51085040e-02,
           1.51085040e-02],
          [9.48149258e-03, 9.48149258e-03, 9.48149258e-03,
           9.48149258e-03],
          [6.78298311e-03, 6.78298311e-03, 6.78298311e-03,
           6.78298311e-03]],

         [[4.71868703e-02, 4.71868703e-02, 4.71868703e-02,
           4.71868703e-02],
          [2.96125917e-02, 2.96125917e-02, 2.96125917e-02,
           2.96125917e-02],
          [2.11846086e-02, 2.11846086e-02, 2.11846086e-02,
           2.11846086e-02]],

         [[9.72619179e-02, 9.72619179e-02, 9.72619179e-02,
           9.72619179e-02],
          [6.10376879e-02, 6.10376879e-02, 6.10376879e-02,
           6.10376879e-02],
          [4.36658683e-02, 4.36658683e-02, 4.36658683e-02,
           4.36658683e-02]]],


        [[[2.89214956e-02, 2.89214956e-02, 2.89214956e-02,
           2.89214956e-02],
          [1.81499734e-02, 1.81499734e-02, 1.81499734e-02,
           1.81499734e-02],
          [1.29843442e-02, 1.298434

In [74]:
def computePartialFrequencyMatrix(utilities, frequencies, vehicles, tasks, v):
    """
        Compute the expected utility for vehicle v foreach task, considering for each 
        other vehicle a random allocation choice with probability equal to empirical frequency observed 
        
    """
    temp_vehicles = vehicles[:v] + vehicles[v+1:] # create a list of index without vehicle v
    temp_alloc = np.zeros([len(tasks)]*(len(vehicles)-1)) # create a 0 array of shape (vehicles-1)^task
    allAlloc = [x for x,_ in np.ndenumerate(temp_alloc)] # enumerate all possible allocations for these vehicles
    expectations = []
    for t in range(len(tasks)):
        expected = 0
        for alloc in allAlloc : # for each possible allocation for vehicles without v
            # compute proba for these vehicles to do this allocation : 
            proba = np.prod([frequencies[temp_vehicles[v]][alloc[v]] for v in range(len(temp_vehicles))])
            # recreating full allocation with v :
            index = [0]*len(vehicles)
            for i in range(len(temp_vehicles)) :
                index[temp_vehicles[i]] = alloc[i]
            index[v] = t
            # get the utility for v if it do task t with this allocation for the other vehicles
            utility = utilities[tuple(index)][v]
            expected+= utility * proba # add proba time the utility of v to the expectation
        expectations.append(expected)
    return expectations

In [71]:
# testing expectation computation
frequencies = np.random.random((4,10)) # random frequencies (note that sum != 1, it's just for example)
frequencies

array([[0.21009582, 0.11943391, 0.7108427 , 0.91304851, 0.02249661,
        0.87766122, 0.39696832, 0.7457213 , 0.16379217, 0.54709451],
       [0.63362027, 0.12801527, 0.50893712, 0.73058241, 0.46700579,
        0.74985575, 0.15322143, 0.97573095, 0.29776437, 0.45045402],
       [0.71731982, 0.18131516, 0.91146534, 0.44454428, 0.84013496,
        0.36539865, 0.48351168, 0.85432586, 0.06412623, 0.88914702],
       [0.03052541, 0.86650322, 0.92761635, 0.43953926, 0.96805666,
        0.27068533, 0.88673364, 0.12493805, 0.74106153, 0.07182016]])

In [76]:
for v in vehicles : # get the task with the highest utility expectation for each vehicle
    expect = computePartialFrequencyMatrix(ut, frequencies, vehicles, tasks, v)
    print(str(expect)+" --> task "+str(np.argmax(expect)))

[19.52697896567259, 18.730815737618343, 24.867858909366248] --> task 2
[16.5332330478646, 20.729079619506614, 16.17518652391417] --> task 1
[12.996964693052485, 14.300402422881186, 12.27633887531842] --> task 1
[10.923915888343553, 11.52656292187575, 7.27074793265533] --> task 1


In [215]:
def fictitiousPlay(utilities, vehicles, tasks, maxsteps):
    """ Play Fictitious Play until maxsteps
    
    Parameters :
        utilities : Matrix(int) the utility matrix of dimension nb_tasks^nb_vehicles * nb_vehicles
        vehicles : List(int), the list of vehicle ids
        tasks : List(int) the list of task ids
        maxteps : int, the steps limit of the algorithm
        
    Returns : 
        List(int), the fond allocation after maxsteps steps (no optimality guarantee)
        
    """
    passedPropositions = [[t for t in tasks] for v in vehicles]
    steps = 0
    while steps < maxsteps:
        # vehicle id_change has interest to change to a better allocation
        for v in vehicles :
            f = [computeFrequency(passedPropositions[v],tasks) for v in vehicles]
            expect = computePartialFrequencyMatrix(utilities, f, vehicles, tasks, v)
            bestTask = np.argmax(expect)
            passedPropositions[v].append(bestTask)
        #print([passedPropositions[v][-1] for v in vehicles])
        steps += 1
    return [passedPropositions[v][-1] for v in vehicles]

In [213]:
# testing FP compared to BRD on some examples :

In [204]:
bestResponseDynamic(ut, vehicles, tasks, 1000)

[0, 0, 2, 1]

In [205]:
bestResponseDynamic(ut, vehicles, tasks, 1000)

[2, 0, 1, 2]

In [206]:
bestResponseDynamic(ut, vehicles, tasks, 1000)

[1, 2, 0, 0]

In [207]:
fictitiousPlay(ut, vehicles, tasks, 1000)

[2, 1, 2, 0]

In [212]:
fictitiousPlay(ut, vehicles, tasks, 1000) # this is a better solution than BRD's NE in term of social utility

[2, 1, 1, 0]

In [200]:
# obtained with fictitious play : 
# [2,1,1,0]  (maxsteps 500)
# [2,1,2,0]  (maxsteps 1500)
print("FP : "+str(ut[2,1,1,0])+" -> mean "+str(np.mean(ut[2,1,1,0])))
print("FP : "+str(ut[2,1,2,0])+" -> mean "+str(np.mean(ut[2,1,2,0])))

# obtained with best response dynamic : 
# [1,2,0,0]  (maxsteps 1000)
# [2,0,1,2]  (maxsteps 1000)
# [0,0,2,1]  (maxsteps 1000)

print("BRD : "+str(ut[1,2,0,0])+" -> mean "+str(np.mean(ut[1,2,0,0])))
print("BRD : "+str(ut[2,0,1,2])+" -> mean "+str(np.mean(ut[2,0,1,2])))
print("BRD : "+str(ut[0,0,2,1])+" -> mean "+str(np.mean(ut[0,0,2,1])))

FP : [8 9 6 7] -> mean 7.5
FP : [2 8 6 6] -> mean 5.5
BRD : [9 6 8 5] -> mean 7.0
BRD : [5 3 1 9] -> mean 4.5
BRD : [6 9 5 8] -> mean 7.0
